In [10]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Set up the WebDriver (use the path to your driver)
driver = webdriver.Chrome(executable_path='C:/chrome-win64')


# Open a website
driver.get("https://www.google.com")

# Close the browser
driver.quit()

# # URL for the 2025 NBA season game results
# url = 'https://www.basketball-reference.com/leagues/NBA_2025_games.html'

# # Open the webpage
# driver.get(url)

# # Wait for the page to load completely (adjust the time if needed)
# time.sleep(5)

# # Find the table containing the game results
# table = driver.find_element(By.CLASS_NAME, 'sortable.stats_table')

# # Extract the table headers
# headers = [th.text.strip() for th in table.find_elements(By.TAG_NAME, 'th')]

# # Extract all the rows in the table
# rows = table.find_elements(By.TAG_NAME, 'tr')[1:]  # Skip the header row

# # Extract data from each row
# game_data = []
# for row in rows:
#     cols = row.find_elements(By.TAG_NAME, 'td')
#     game_data.append([ele.text.strip() for ele in cols])

# # Create a DataFrame from the extracted data
# df = pd.DataFrame(game_data, columns=headers[1:])  # Skip first column 'Rk'

# # Clean up the DataFrame (remove any empty rows or columns)
# df = df.dropna()

# # Close the browser after scraping
# driver.quit()

# # Display the DataFrame
# print(df.head())


TypeError: WebDriver.__init__() got an unexpected keyword argument 'executable_path'